In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
#from scipy.stats import norm
# from scipy import stats
# from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
non_coding_region="core_prom"
core_prom_type= "NonTATA"
cancer_type= "Brain"

In [3]:
base_dir = "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/DNABERT_Data/{}/Patient_wise".format(cancer_type, non_coding_region)
output_dir= "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/".format(cancer_type, non_coding_region)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
df_core_prom = pd.read_csv("/home/pdutta/Data/Noncoding_region/core_prom_bed.tsv", sep="\t")
df_core_prom

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr7,ENSG00000004059.11,ENST00000000233.10,ENSE00001872691.2,+,127588411,127588366,127588455
1,chr7,ENSG00000001626.16,ENST00000003084.11,ENSE00001343851.2,+,117480025,117479980,117480069
2,chr7,ENSG00000001630.17,ENST00000003100.13,ENSE00001216550.10,-,92134477,92134433,92134522
3,chr7,ENSG00000002933.9,ENST00000004103.8,ENSE00001827596.2,+,150800769,150800724,150800813
4,chr7,ENSG00000004799.8,ENST00000005178.6,ENSE00001133344.6,-,95596516,95596472,95596561
...,...,...,...,...,...,...,...,...
236970,chr15,ENSG00000103876.14,ENST00000684569.1,ENSE00003921718.1,+,80153010,80152965,80153054
236971,chr15,ENSG00000213614.11,ENST00000684602.1,ENSE00002620879.1,-,72375989,72375945,72376034
236972,chr15,ENSG00000213614.11,ENST00000684667.1,ENSE00002584031.1,-,72375972,72375928,72376017
236973,chr15,ENSG00000104133.16,ENST00000684676.1,ENSE00003922470.1,-,44663688,44663644,44663733


In [5]:
with open("/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Intersected_Data/intersected_vcf_{}_data.pkl".format(non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [6]:
len(loaded_dictionary)

432

In [7]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [8]:
patient_dfs= []
patient_id = []
for dirpath, dirnames, filenames in os.walk(base_dir):
    #print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        print(dir_name)
        #input()
        df_temp = loaded_dictionary[dir_name]
        print(len(df_temp))
        df_temp = df_temp.drop_duplicates(subset=['chr_name', 'start', 'end', 'transcript_id','START_POS', 'END_POS', 'REF','ALT']).reset_index()
        concat_probab = np.load(dir_path+"/{}/Prediction_result/pred_results.npy".format(core_prom_type))
        print(len(concat_probab))
        print(len(df_temp))
        if(2*len(df_temp)==len(concat_probab)):
            #input()
            Reference_probab = concat_probab[::2]
            Alternative_probab = concat_probab[1::2]
            df_temp['Ref_probab']=Reference_probab
            df_temp['Alt_probab']=Alternative_probab
            df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
            df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
            #display(df_temp)
            #input()
            # Apply the filter conditions
            df_temp = df_temp[((df_temp['Ref_probab'] > 0.5) & (df_temp['Alt_probab'] <= 0.5)) | 
                      ((df_temp['Ref_probab'] <= 0.5) & (df_temp['Alt_probab'] > 0.5))]
            #display(df_temp)
            #input()
            patient_dfs.append(df_temp)
        else:
            print(dir_name)
            input()
        #input()
    break

/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_raw_pindel
a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_raw_pindel
4395
8790
4395
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_raw_pindel
0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_raw_pindel
4155
8310
4155
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan
3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan
4282
8564
4282
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_raw_pindel
af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_raw_pindel
4419
8838
4419
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/67c83385-8497-4c2d-9

FileNotFoundError: [Errno 2] No such file or directory: '/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/dd045b58-4d6a-4404-a412-6aa9e99e282e_sanger_raw_pindel/NonTATA/Prediction_result/pred_results.npy'

In [13]:
patient_dfs[0]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
139,183,chr7,18027802,18027891,ENSG00000229937.9,ENST00000506618.5,ENSE00002050515.4,-,18027846,chr7,...,D,18027861,2,18027856,80,1336.72998046875,0.596329,0.437189,-0.094900,0.927321
156,205,chr7,129953189,129953278,ENSG00000244036.3,ENST00000587038.2,ENSE00002737223.1,+,129953234,chr7,...,DI,129953233,0,129953228,208,.,0.180505,0.531642,0.186679,-2.365540
310,398,chr8,117520668,117520757,ENSG00000164758.7,ENST00000297347.7,ENSE00001087222.3,+,117520713,chr8,...,D,117520716,1,117520714,66,1261.449951171875,0.265896,0.599635,0.200122,-2.047891
322,414,chr8,74376642,74376731,ENSG00000207417.1,ENST00000384686.1,ENSE00001499694.1,+,74376687,chr8,...,D,74376675,1,74376666,20,595.7000122070312,0.326878,0.682859,0.243084,-2.148573
327,419,chr8,123737349,123737438,ENSG00000104537.17,ENST00000419625.6,ENSE00001292122.3,-,123737393,chr8,...,D,123737418,6,123737411,299,2874.739990234375,0.462950,0.534649,0.038334,-0.414472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3890,5019,chr17,42611830,42611919,ENSG00000131462.9,ENST00000679484.1,ENSE00003915983.1,+,42611875,chr17,...,D,42611847,4,42611837,56,1106.1199951171875,0.488609,0.819456,0.271114,-2.248062
3919,5064,chr18,69956626,69956715,ENSG00000150637.9,ENST00000583955.5,ENSE00002724235.1,-,69956670,chr18,...,D,69956682,1,69956672,140,1874.3699951171875,0.466241,0.573655,0.061619,-0.623276
3939,5094,chr18,109020,109109,ENSG00000263006.6,ENST00000608049.5,ENSE00003709318.1,+,109065,chr18,...,D,109084,1,109082,18,765.364013671875,0.150796,0.792526,0.508587,-4.427033
3965,5125,chr18,58808423,58808512,ENSG00000267593.2,ENST00000659493.1,ENSE00003868021.1,+,58808468,chr18,...,D,58808494,1,58808485,90,1619.050048828125,0.291222,0.886532,0.527762,-4.249107


In [14]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).drop_duplicates(subset=['chr_name', 'transcript_id', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT'])
unique_patient_regions

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
139,183,chr7,18027802,18027891,ENSG00000229937.9,ENST00000506618.5,ENSE00002050515.4,-,18027846,chr7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,205,chr7,129953189,129953278,ENSG00000244036.3,ENST00000587038.2,ENSE00002737223.1,+,129953234,chr7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,398,chr8,117520668,117520757,ENSG00000164758.7,ENST00000297347.7,ENSE00001087222.3,+,117520713,chr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
322,414,chr8,74376642,74376731,ENSG00000207417.1,ENST00000384686.1,ENSE00001499694.1,+,74376687,chr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
327,419,chr8,123737349,123737438,ENSG00000104537.17,ENST00000419625.6,ENSE00001292122.3,-,123737393,chr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2929,4272,chr5,142142933,142143022,ENSG00000131507.11,ENST00000503388.1,ENSE00002049510.1,+,142142978,chr5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4350,6159,chr2,148978330,148978419,ENSG00000168280.18,ENST00000678720.1,ENSE00003908279.1,+,148978375,chr2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4656,6544,chr3,71005292,71005381,ENSG00000114861.23,ENST00000647741.1,ENSE00003839066.1,-,71005336,chr3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,304,chr13,29850573,29850662,ENSG00000122042.10,ENST00000380680.5,ENSE00001485973.5,-,29850617,chr13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
filtered_regions = unique_patient_regions.groupby(['chr_name', 'transcript_id', 'start', 'end']).filter(lambda x: len(x) > 1 and not all(x['START_POS'].eq(x['START_POS'].iloc[0]) & x['END_POS'].eq(x['END_POS'].iloc[0]) & x['REF'].eq(x['REF'].iloc[0]) & x['ALT'].eq(x['ALT'].iloc[0])))
filtered_regions

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
428,559,chr8,47977248,47977337,ENSG00000104738.18,ENST00000648533.1,ENSE00003840279.1,+,47977293,chr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
532,700,chr16,55479953,55480042,ENSG00000087245.13,ENST00000564864.5,ENSE00002596620.1,+,55479998,chr16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1727,2255,chr11,71813415,71813504,ENSG00000248671.8,ENST00000657407.1,ENSE00003865887.1,-,71813459,chr11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2035,2658,chr6,26033574,26033663,ENSG00000278463.2,ENST00000615868.2,ENSE00003740156.2,-,26033618,chr6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2066,2694,chr6,121437337,121437426,ENSG00000152661.9,ENST00000650427.1,ENSE00003838947.1,+,121437382,chr6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,3657,chr4,6673852,6673941,ENSG00000251580.3,ENST00000647523.2,ENSE00003830184.1,-,6673896,chr4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2929,4272,chr5,142142933,142143022,ENSG00000131507.11,ENST00000503388.1,ENSE00002049510.1,+,142142978,chr5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4350,6159,chr2,148978330,148978419,ENSG00000168280.18,ENST00000678720.1,ENSE00003908279.1,+,148978375,chr2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,304,chr13,29850573,29850662,ENSG00000122042.10,ENST00000380680.5,ENSE00001485973.5,-,29850617,chr13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
pd.set_option('display.max_columns', None)
filtered_regions[filtered_regions['start']==47977248]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
428,559,chr8,47977248,47977337,ENSG00000104738.18,ENST00000648533.1,ENSE00003840279.1,+,47977293,chr8,47977262,47977264,2dd1b052-9a77-11ea-8b2d-d98707b0d7b9,CA,C,240.0,FF006,True,1.0,.,D,47977286.0,22.0,47977263.0,8.0,293.4389953613281,0.912657,0.269687,-0.586811,4.822542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,845,chr8,47977248,47977337,ENSG00000104738.18,ENST00000648533.1,ENSE00003840279.1,+,47977293,chr8,47977261,47977263,1aed102a-24b6-11eb-ab83-3c820e73e08a,TC,T,240.0,FF010,True,1.0,.,D,47977264.0,1.0,47977262.0,5.0,332.8659973144531,0.912657,0.284891,-0.572936,4.713073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461,566,chr8,47977248,47977337,ENSG00000104738.18,ENST00000648533.1,ENSE00003840279.1,+,47977293,chr8,47977262,47977266,38f9c22a-97ee-11ea-8129-40efc8ec6102,CAAA,C,180.0,FF006,True,3.0,.,D,47977286.0,22.0,47977263.0,4.0,254.11900329589844,0.912657,0.495189,-0.381005,3.413080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Reset the option to its default (typically 20)
pd.reset_option('display.max_columns')

In [18]:
unique_patient_regions[unique_patient_regions['start']==47977248]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
428,559,chr8,47977248,47977337,ENSG00000104738.18,ENST00000648533.1,ENSE00003840279.1,+,47977293,chr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,845,chr8,47977248,47977337,ENSG00000104738.18,ENST00000648533.1,ENSE00003840279.1,+,47977293,chr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
461,566,chr8,47977248,47977337,ENSG00000104738.18,ENST00000648533.1,ENSE00003840279.1,+,47977293,chr8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df)
    for _, region in unique_patient_regions.iterrows():
        #print(region)
        #input()
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr_name'] == region.chr_name) & (df['start'] == region.start)& (df['end'] == region.end) & (df['START_POS'] == region.START_POS) & (df['END_POS'] == region.END_POS) & (df['REF'] == region.REF) & (df['ALT'] == region.ALT) , score_column]
            #print(score_series)
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'].append(score)

(69, 30)
(75, 33)
(99, 30)
(10, 33)
(64, 30)
(3, 33)
(69, 30)
(5, 33)
(63, 33)
(16, 33)
(13, 33)
(73, 30)
(83, 30)
(68, 30)
(6, 33)
(66, 30)
(67, 30)
(70, 30)
(57, 30)
(47, 33)
(6, 33)
(73, 30)
(69, 30)
(62, 30)
(98, 30)
(13, 33)
(59, 33)
(10, 33)
(80, 30)
(70, 30)
(79, 30)
(76, 30)
(85, 30)
(81, 30)
(11, 33)
(76, 33)
(12, 33)
(63, 30)
(5, 33)
(8, 33)
(60, 30)
(72, 30)
(70, 30)
(66, 30)
(9, 33)
(50, 33)
(12, 33)
(74, 30)
(71, 30)
(6, 33)
(73, 30)
(9, 33)
(73, 30)
(5, 33)
(7, 33)
(65, 30)
(70, 30)
(67, 30)
(6, 33)
(67, 30)
(2, 33)
(51, 33)
(7, 33)
(64, 30)
(15, 33)
(10, 33)
(52, 33)
(74, 30)
(50, 33)
(76, 30)
(9, 33)
(74, 30)
(9, 33)
(13, 33)
(4, 33)
(7, 33)
(63, 30)
(4, 33)
(86, 30)
(77, 30)
(11, 33)
(10, 33)
(4, 33)
(23, 33)
(60, 30)
(57, 30)
(81, 30)
(78, 30)
(5, 33)
(8, 33)
(11, 33)
(95, 30)
(68, 30)
(73, 30)
(5, 33)
(64, 30)
(75, 33)
(61, 30)
(68, 30)
(69, 30)
(3, 33)
(5, 33)
(12, 33)
(3, 33)
(8, 33)
(34, 33)
(63, 30)
(11, 33)
(95, 30)
(68, 30)


In [20]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

ValueError: All arrays must be of the same length

In [ ]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

In [ ]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

In [ ]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

In [ ]:
result_df_ref.to_csv(output_dir+"df_ref_score.tsv", sep="\t", index=False)
result_df_alt.to_csv(output_dir+"df_alt_score.tsv", sep="\t", index=False)
result_df_log_odd.to_csv(output_dir+"df_log_odd_score.tsv", sep="\t", index=False)
result_df_score_change.to_csv(output_dir+"df_score_change.tsv", sep="\t", index=False)